In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from functions import *

url = "http://www.cinemaonline.sg/movies/nowshowing.aspx"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")
tags = soup.findAll("li")
tags = [str(i) for i in tags]

movieData = []
for i in range(0, len(tags)):
    movieData.append(striphtml(tags[i]))

del movieData[:20]
movieData = movieData[:len(movieData)-4]

for i in range(0, len(movieData)):
    movieData[i] = stripStuff(movieData[i])

duration = []
movieName = []
classification = []
castList = []
releaseDate = []

for i in movieData:
    find = re.search("Running Time: (.+?)Release", i).group(1)
    duration.append(find)
    find = re.search("(.+?)\(", i).group(1)
    movieName.append(find)
    find = re.search("Classification: (.+?)Genre:", i).group(1)
    classification.append(find)
    find = re.search("Cast: (.+?)\r", i).group(1)
    castList.append(find)
    find = re.search("Release Date: (.+?)Language", i).group(1)
    releaseDate.append(find)
    
for i in range(0, len(movieName)):
    movieName[i] = stripWhitespace(movieName[i])

ID = []

for i in range(1, len(movieData)+1):
    if i<10:
        ID.append("1000" + str(i))
    else:
        ID.append("100" + str(i))

viewType = []
viewStatus = []
time = []

for i in range(0, len(movieName)):
    viewType.append("Digital")
    viewStatus.append("Now Showing")
    time.append(datetime.now())

movieDF = pd.DataFrame(np.column_stack([movieName, castList, duration, releaseDate, classification, viewType, viewStatus, time]), 
                            columns = ['Name', 'Casts', 'Runtime', 'ReleaseDate', 'Classification', 'ViewType', 'ViewStatus', 'IngestionTime'])

In [2]:
movieDF

,Name,Casts,Runtime,ReleaseDate,Classification,ViewType,ViewStatus,IngestionTime
0,The Vanished,"Thomas Jane, Anne Heche, Jason Patric",1 Hour 55 Minutes,16 Sep 2021,NC16,Digital,Now Showing,2021-09-17 11:38:55.902532
1,Time,"Patrick Tse, Fung Bo Bo, Lam Suet, Sam Lee",1 Hour 38 Minutes,16 Sep 2021,PG13,Digital,Now Showing,2021-09-17 11:38:55.902532
2,Liverpool FC: The End Of The Storm,"Trent Alexander-Arnold, Alisson Becker, Kenny ...",1 Hour 39 Minutes,16 Sep 2021,PG,Digital,Now Showing,2021-09-17 11:38:55.902532
3,Occupation: Rainfall,"Dan Ewing, Temuera Morrison, Daniel Gillies, L...",2 Hours 8 Minutes,16 Sep 2021,PG13,Digital,Now Showing,2021-09-17 11:38:55.902532
4,Dune,"Timothée Chalamet, Rebecca Ferguson, Oscar Isa...",2 Hours 35 Minutes,16 Sep 2021,PG13,Digital,Now Showing,2021-09-17 11:38:55.902532
5,Escape Room 2: Tournament Of Champions,"Taylor Russell, Logan Miller, Isabelle Fuhrman...",1 Hour 28 Minutes,16 Sep 2021,PG13,Digital,Now Showing,2021-09-17 11:38:55.902532
6,Thalaivi,"Kangana Ranaut, Arvind Swamy, Prakash Raj",2 Hours 30 Minutes,10 Sep 2021,PG,Digital,Now Showing,2021-09-17 11:38:55.902532
7,Thalaivi,"Kangana Ranaut, Arvind Swamy, Prakash Raj",2 Hours 33 Minutes,10 Sep 2021,PG,Digital,Now Showing,2021-09-17 11:38:55.902532
8,Malignant,"Annabelle Wallis, Jake Abel, George Young, Jac...",1 Hour 51 Minutes,09 Sep 2021,M18,Digital,Now Showing,2021-09-17 11:38:55.902532
9,Queenpins,"Kristen Bell, Kirby Howell-Baptiste, Paul Walt...",1 Hour 50 Minutes,09 Sep 2021,NC16,Digital,Now Showing,2021-09-17 11:38:55.902532
